In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://nbviewer.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
! ls /kaggle/input/ktrain-library/ktrain

!pip install facenet-pytorch --no-index --find-links=file:///kaggle/input/facenet_pytorch/ 

In [ ]:
!pip install ktrain --no-index --find-links=file:///kaggle/input/ktrain-library/ktrain

In [ ]:
#!pip install ktrain

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import tensorflow as tf

In [ ]:
import seaborn as sns

In [ ]:
import ktrain

In [ ]:
from ktrain import text

Tweak for no internet acces and bigru

In [ ]:
!mkdir /root/ktrain_data

In [ ]:
cp -v ../input/fasttext-pretrainedvectors-english-text/cc.en.300.vec  /root/ktrain_data/

Load training dataset validation score 0.665

In [ ]:
#df=pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
#df=pd.read_csv("/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/train.csv")
#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/test_labels.csv

#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/test.csv

df=pd.read_csv("../input/jigsaw-toxic-coomens-annotations/jigsaw_train_hate_annotationprob.csv",index_col=0)

In [ ]:
df.head()

Load evaluation dataset

In [ ]:
df_valid=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)

Load comments to score

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)

**Test Mode**

For testing purpose

In [ ]:
mode='prod'
#mode='test'
sample=0.01
if mode=='test':
    df=df.sample(frac=sample)
if mode=='test':
    df_valid=df_valid.sample(frac=sample)
if mode=='test':
    comments_to_score=comments_to_score.sample(frac=sample)

undersampling

In [ ]:
class_1,class_2 = df.toxic.value_counts()
print(class_1,class_2)
c2 = df[df["toxic"] >0]
c1 = df[df.toxic == 0]
print(len(c1),len(c2))
df_2 = c1.sample(class_2)
print(len(df_2))
undersampled_df = pd.concat([df_2,c2],axis=0)
undersampled_df.drop_duplicates(subset ="text",keep = False, inplace = True)
df=undersampled_df

In [ ]:
one_hot = pd.get_dummies(df['hateoffensive_class'])

In [ ]:
one_hot.head()

In [ ]:
df = df.drop('hateoffensive_class',axis = 1)
# Join the encoded df
df = df.join(one_hot)

In [ ]:
df.columns

In [ ]:
df = df[['text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','hate_speech', 'neutral','offensive']]

In [ ]:
df.head()

Bert model classification

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.25)
#test, valid = train_test_split(test, test_size=0.05)

In [ ]:
#train
x_train =train.text.values
x_test = test.text.values
#validation

In [ ]:
y_train=train.loc[:,"toxic":"offensive"]
y_train=y_train.values.tolist()
y_test=test.loc[:,"toxic":"offensive"]
y_test=y_test.values.tolist()

In [ ]:
class_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','hate_speech', 'neutral','offensive']

In [ ]:
#MODEL_NAME = 'distilbert-base-uncased'
MODEL_NAME = '../input/huggingface-bert/bert-base-uncased/'
#MODEL_NAME = '../input/roberta-base/'
t = text.Transformer(MODEL_NAME, maxlen=230,class_names=class_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

In [ ]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','hate_speech', 'neutral','offensive']

In [ ]:
bert_model=t.get_classifier()

thanks to amaya
https://github.com/amaiya/ktrain/blob/master/FAQ.md

* import ktrain
* from ktrain import text
* t = text.Transformer('/tmp/my_model_files', maxlen=500, class_names=label_list)
* trn = t.preprocess_train(x_train, y_train)
* model = t.get_classifier()
* learner = ktrain.get_learner(model, train_data=trn, batch_size=8)
* learner.fit_onecycle(5e-5, 1)

In [ ]:
bert_learner = ktrain.get_learner(bert_model, train_data=trn, val_data=val,batch_size=8)
#learner.fit_onecycle(5e-5, 2)

In [ ]:
# # define a custom callback for ROC-AUC
# from tensorflow.keras.callbacks import Callback
# from sklearn.metrics import roc_auc_score
# class RocAucEvaluation(Callback):
#     def __init__(self, validation_data=(), interval=1):
#         super(Callback, self).__init__()

#         self.interval = interval
#         self.X_val, self.y_val = validation_data

#     def on_epoch_end(self, epoch, logs={}):
#         if epoch % self.interval == 0:
#             y_pred = self.model.predict(self.X_val, verbose=0)
#             score = roc_auc_score(self.y_val, y_pred)
#             print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
# RocAuc = RocAucEvaluation(validation_data=(x_test, y_test), interval=1)

In [ ]:
bert_learner.autofit(4e-5, 3)
#learner.autofit(0.001, 2, callbacks=[RocAuc])

In [ ]:
bert_learner.plot('loss')

In [ ]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','hate_speech', 'neutral','offensive']
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','hate_speech', 'neutral','offensive']

In [ ]:
bert_predictor = ktrain.get_predictor(bert_learner.model, preproc=t)

In [ ]:
def adjust_score(x):
    if x>0:
        return x+2
    else:
        return x
def scoring_preds2(X_less_toxic):
    label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','hate_speech', 'neutral','offensive']
    less_toxic_df=pd.DataFrame()
    left_toxic=bert_predictor.predict_proba(X_less_toxic)
    less_toxic_df['preds']=left_toxic.tolist()
    temp_df=pd.DataFrame(less_toxic_df['preds'].to_list(), columns=label_columns)
    less_toxic_df['toxic']=temp_df['toxic']
    less_toxic_df['severe_toxic']=temp_df['severe_toxic']
    less_toxic_df['obscene']=temp_df['obscene']
    less_toxic_df['threat']=temp_df['threat']
    less_toxic_df['insult']=temp_df['insult']
    less_toxic_df['identity_hate']=temp_df['identity_hate']
    less_toxic_df['neutral']=temp_df['neutral']
    less_toxic_df['offensive']=temp_df['offensive']
    less_toxic_df['hate_speech']=temp_df['hate_speech']
    #less_toxic_df['neutral_mask']=less_toxic_df['toxic'].apply(lambda x: neutral_mask(x))
    w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    # #w1,w2,w3,w4,w5,w6=[0.48841081, 0.21194874, 0.39584097, 0.40030452, 0.46899059, 0.42384744]
    less_toxic_df['score1']=w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    w1,w2,w3,w4,w5,w6=[1, 1, 1, 1, 1, 1]
    less_toxic_df['score2']=w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    w1,w2,w3,w4,w5,w6=[0.48841081, 0.21194874, 0.39584097, 0.40030452, 0.46899059, 0.42384744]
    less_toxic_df['score3']=1-less_toxic_df["neutral"]+less_toxic_df["hate_speech"]+less_toxic_df["hate_speech"]+w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #less_toxic_df['score']= less_toxic_df['neutral_mask']*less_toxic_df['score']+less_toxic_df['toxic']
    w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    less_toxic_df['score4']=1-less_toxic_df["neutral"]+1*less_toxic_df['obscene']+1*less_toxic_df['toxic']+1*less_toxic_df['threat']+1*less_toxic_df['insult']+2*less_toxic_df['severe_toxic']+1.5*less_toxic_df['identity_hate']
    w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    less_toxic_df['score5']=1-less_toxic_df["neutral"]+1*less_toxic_df['obscene']+1*less_toxic_df['toxic']+1*less_toxic_df['threat']+1*less_toxic_df['insult']+2*less_toxic_df['severe_toxic']+less_toxic_df['identity_hate']
    #df['score2']=1-jigsaw_df["neutral_speech_agreement"]+1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+2*df['severe_toxic']+1*df['identity_hate']
    w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    less_toxic_df['score6']=1-less_toxic_df["neutral"]+w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #df['score2']=1-jigsaw_df["neutral_speech_agreement"]+1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+2*df['severe_toxic']+1*df['identity_hate']
    return less_toxic_df

In [ ]:
#validation
X_less_toxic =df_valid.less_toxic.values
X_more_toxic =df_valid.more_toxic.values
#submit file
#comments_to_score=comments_to_score.sample(100)
X_comments_to_score =comments_to_score.text.values

In [ ]:
less_toxic2= scoring_preds2(X_less_toxic)
more_toxic2= scoring_preds2(X_more_toxic)

p3=less_toxic2['score1'].values
p4=more_toxic2['score1'].values
(p3 < p4).mean()

In [ ]:
p5=less_toxic2['score2'].values
p6=more_toxic2['score2'].values
print('scorul pentru formula 2 este ',(p5 < p6).mean())

In [ ]:
p7=less_toxic2['score3'].values
p8=more_toxic2['score3'].values
print('scorul pentru formula 3 este ',(p7 < p8).mean())

In [ ]:
p9=less_toxic2['score4'].values
p10=more_toxic2['score4'].values
print('scorul pentru formula 4 este ',(p9 < p10).mean())

In [ ]:
p11=less_toxic2['score5'].values
p12=more_toxic2['score5'].values
print('scorul pentru formula 5 este ',(p11 < p12).mean())

In [ ]:
p13=less_toxic2['score6'].values
p14=more_toxic2['score6'].values
print('scorul pentru formula 6 este ',(p13 < p14).mean())

In [ ]:
comments_to_score_preds=scoring_preds2(X_comments_to_score)

In [ ]:
#scorul pentru formula 1 este 0.6910299003322259
#scorul pentru formula 2 este  0.6644518272425249
#scorul pentru formula 3 este  0.6744186046511628
#scorul pentru formula 4 este  0.6810631229235881
#scorul pentru formula 5 este  0.6744186046511628
#scorul pentru formula 6 este  0.6976744186046512
scores_model1=comments_to_score_preds['score1']
scores_model2=comments_to_score_preds['score2']
scores_model3=comments_to_score_preds['score3']
scores_model4=comments_to_score_preds['score4']
scores_model5=comments_to_score_preds['score5']
scores_model6=comments_to_score_preds['score6']

Slim Model

In [ ]:
y_train=train.loc[:,"toxic":"identity_hate"]
y_train=y_train.values.tolist()
y_test=test.loc[:,"toxic":"identity_hate"]
y_test=y_test.values.tolist()

In [ ]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

In [ ]:
bert_model2=t.get_classifier()

In [ ]:
bert_learner2 = ktrain.get_learner(bert_model2, train_data=trn, val_data=val,batch_size=8)
#learner.fit_onecycle(5e-5, 2)

In [ ]:
# train
bert_learner2.autofit(4e-5, 3)

In [ ]:
bert_learner2.plot('loss')

In [ ]:
bert_predictor2 = ktrain.get_predictor(bert_learner2.model, preproc=t)

In [ ]:
ef scoring_preds1(X_less_toxic):
    label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    less_toxic_df=pd.DataFrame()
    left_toxic=bert_predictor.predict_proba(X_less_toxic)
    less_toxic_df['preds']=left_toxic.tolist()
    temp_df=pd.DataFrame(less_toxic_df['preds'].to_list(), columns=label_columns)
    less_toxic_df['toxic']=temp_df['toxic']
    less_toxic_df['severe_toxic']=temp_df['severe_toxic']
    less_toxic_df['obscene']=temp_df['obscene']
    less_toxic_df['threat']=temp_df['threat']
    less_toxic_df['insult']=temp_df['insult']
    less_toxic_df['identity_hate']=temp_df['identity_hate']
    less_toxic_df['neutral_mask']=less_toxic_df['toxic'].apply(lambda x: neutral_mask(x))
    w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    # #w1,w2,w3,w4,w5,w6=[0.48841081, 0.21194874, 0.39584097, 0.40030452, 0.46899059, 0.42384744]
    less_toxic_df['score1']=w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    w1,w2,w3,w4,w5,w6=[1, 2, 1, 1, 1, 1]
    less_toxic_df['score2']=w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
    w1,w2,w3,w4,w5,w6=[0.48841081, 0.21194874, 0.39584097, 0.40030452, 0.46899059, 0.42384744]
    less_toxic_df['score3']=w1*less_toxic_df['toxic']+w2*less_toxic_df['severe_toxic']+w3*less_toxic_df['obscene']+w4*less_toxic_df['threat']+w5*less_toxic_df['insult']+w6*less_toxic_df['identity_hate']
    #less_toxic_df['score']= less_toxic_df['neutral_mask']*less_toxic_df['score']+less_toxic_df['toxic']
    return less_toxic_df

In [ ]:
less_toxic2= scoring_preds1(X_less_toxic)
more_toxic2= scoring_preds1(X_more_toxic)

p3=less_toxic2['score1'].values
p4=more_toxic2['score1'].values
(p3 < p4).mean()

In [ ]:
p5=less_toxic2['score2'].values
p6=more_toxic2['score2'].values
(p3 < p4).mean()

In [ ]:
p5=less_toxic2['score2'].values
p6=more_toxic2['score2'].values
(p3 < p4).mean()

In [ ]:
comments_to_score_df=scoring_preds1(X_comments_to_score)

In [ ]:
scores_model9=comments_to_score_df['score1'].values

**2nd Model**

In [ ]:
df.head()

In [ ]:
w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
#w1,w2,w3,w4,w5,w6=[0.48841081, 0.21194874, 0.39584097, 0.40030452, 0.46899059, 0.42384744]
df['score']=w1*df['toxic']+w2*df['severe_toxic']+w3*df['obscene']+w4*df['threat']+w5*df['insult']+w6*df['identity_hate']

In [ ]:
del x_train 
del y_train
del x_test 
del y_test 

In [ ]:
train, test = train_test_split(df, test_size=0.25)
#train
x_train =train.text.values
y_train = train.score.values
x_test = test.text.values
y_test = test.score.values
#validation
X_less_toxic =df_valid.less_toxic.values
X_more_toxic =df_valid.more_toxic.values
#submit file
X_comments_to_score =comments_to_score.text.values

In [ ]:
trn, val, preproc2 = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          ngram_range=1, 
                                          maxlen=300, 
                                          max_features=50000)

In [ ]:
model3 = text.text_regression_model('bigru', train_data=trn, preproc=preproc2)
learner3 = ktrain.get_learner(model3, train_data=trn, val_data=val, batch_size=32)

In [ ]:
#learner.autofit(4e-4, 8)
learner3.autofit(5e-5, 8)

In [ ]:
learner3.plot('loss')

In [ ]:
learner3.fit_onecycle(2e-4, 5)

In [ ]:
learner3.plot('loss')

In [ ]:
learner3.autofit(5e-5, 2)

In [ ]:
learner3.plot('loss')

In [ ]:
predictor3 = ktrain.get_predictor(learner3.model, preproc2)

In [ ]:
p1 = predictor3.predict(X_less_toxic)
p2 = predictor3.predict(X_more_toxic)
# Validation Accuracy
(p1 < p2).mean()

In [ ]:
scores_model7=predictor3.predict(X_comments_to_score)

**3rd model**

In [ ]:
# del x_train 
# del y_train
# del x_test 
# del y_test 

Blended Score

1. lista scoruri model 1 = scores_model1
1. lista scoruri model 2 = scores_model2
1. lista scoruri model 3= predictor.predict(X_more_toxic)


In [ ]:
from scipy.stats import rankdata

In [ ]:
comments_to_score['s1']=scores_model1.values
comments_to_score['rank1']=rankdata( comments_to_score['s1'], method='ordinal')

In [ ]:
comments_to_score['s2']=scores_model2.values
comments_to_score['rank2']=rankdata( comments_to_score['s2'], method='ordinal')

In [ ]:
comments_to_score['s3']=scores_model3.values
comments_to_score['rank3']=rankdata( comments_to_score['s3'], method='ordinal')

In [ ]:
comments_to_score['s4']=scores_model4.values
comments_to_score['rank4']=rankdata( comments_to_score['s4'], method='ordinal')

In [ ]:
comments_to_score['s5']=scores_model5.values
comments_to_score['rank5']=rankdata( comments_to_score['s5'], method='ordinal')

In [ ]:
comments_to_score['s6']=scores_model6.values
comments_to_score['rank6']=rankdata( comments_to_score['s6'], method='ordinal')

In [ ]:
comments_to_score['s7']=scores_model7
comments_to_score['rank7']=rankdata( comments_to_score['s7'], method='ordinal')

In [ ]:
comments_to_score['s8']=scores_model9
comments_to_score['rank8']=rankdata( comments_to_score['s8'], method='ordinal')

#scores_model9

In [ ]:
comments_to_score_preds8

In [ ]:
#comments_to_score['blended_rank']=(4*comments_to_score['rank3']+2*comments_to_score['rank2']+1*comments_to_score['rank1']+1*comments_to_score['rank4'])/8
z1,z2,z3,z4,z5,z6,z7=[1,1,1,1,1,1,1]
#comments_to_score['blended_rank']=(z1*comments_to_score['rank1']+z2*comments_to_score['rank2']+z3*comments_to_score['rank3']+z4*comments_to_score['rank4']+z5*comments_to_score['rank5']+z6*comments_to_score['rank6']+z7*comments_to_score['rank7'])/7
#fara regresie scorul de mai jos
comments_to_score['blended_rank']=(z1*comments_to_score['rank1']+z2*comments_to_score['rank2']+z3*comments_to_score['rank3']+z4*comments_to_score['rank4']+z5*comments_to_score['rank5']+z6*comments_to_score['rank6']+z7*comments_to_score['rank7']+3*comments_to_score['rank8'])/10
comments_to_score['score']=rankdata( comments_to_score['blended_rank'], method='ordinal')


In [ ]:
#comments_to_score['score']=scores
submission=comments_to_score[['score']]
submission.to_csv('submission.csv')
submission.head()

In [ ]:
comments_to_score.head()

In [ ]:
comments_to_score.tail()

In [ ]:
comments_to_score.columns

In [ ]:
rank_df=comments_to_score[['rank1', 'rank2',  'rank3', 'rank4', 'rank5',  'rank6',  'rank7','blended_rank', 'score', ]]

In [ ]:
rank_df.head(10)